### 1. Make Connection to neo4j DB

You can use Neo4j sandbox https://neo4j.com/sandbox/

In [ ]:
from neo4j import GraphDatabase
# Connection parameters
uri = "bolt://localhost:7687"
user = "neo4j"
pwd = "<pwd>"
NEO4J_DATABASE = "dptm-nist-map"

#uri = "bolt://98.84.42.72"
#user = "neo4j"
#pwd = "<pwd>"
#NEO4J_DATABASE = "neo4j"

def create_session(uri, user, pwd):
    driver = GraphDatabase.driver(uri, database=NEO4J_DATABASE, auth=(user, pwd))
    return driver.session()

def execute_query(session, query):
    with session.begin_transaction() as tx:
        tx.run(query)
        tx.commit()  # Ensure changes are committed

# Create a session
session = create_session(uri, user, pwd)
# :style reset - on the browser
# MATCH (n) DETACH DELETE n

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

### 2. Read both DPTM Requirement and NIST Framework

In [ ]:
fw_file = "NIST-Privacy-Framework-structured-withcontextFWReq-withEmbedding.csv"
reg_file = "DPTM_Checklist_Full-withEmbedding.csv"
df_fw = pd.read_csv(fw_file)
df_fw.drop(labels=["Unnamed: 0"],axis=1,inplace=True)

df_reg = pd.read_csv(reg_file)
df_reg.drop(labels=["Unnamed: 0"],axis=1,inplace=True)
#df_reg.columns = ['Function', 'Category', 'Stakeholder', 'Requirement Description', 'Policy Documents', 'Requirement Description Embedding']
df_reg.columns = ['Function', 'Category', 'Requirement Description', 'Policy Type', 'Policy Documents', 'Requirement Description Embedding']

In [ ]:
df_reg.head()

### 3. Code to load both Regulatory Requirements & Framework Requirements into Neo4j

In [ ]:
def create_code(text):
    code_string =  ''.join([string[:2] for string in text.split(' ')]).upper()
    return code_string

def create_reg_code(row):
    code_string =  row['Function Code']+"-"+row['Category Code']+"-"+str(row.name)
    return code_string

In [ ]:
df_reg["Function Code"] = df_reg["Function"].apply(create_code)
df_reg["Category Code"] = df_reg["Category"].apply(create_code)
df_reg["Requirement Code"] = df_reg.apply(create_reg_code,axis=1)

In [ ]:
#create document node. Making both NIST and Regulation as Document nodes
create_doc_nodes = """
CREATE (:Document {Description: "NIST Privacy Framework", Year: 2024, Validity: True}),
       (:Document {Description: "DPTM Checklist", Year: 2024, Validity: True})
"""
# Execute the queries
execute_query(session, create_doc_nodes)

In [ ]:
def create_node(nodetype, code, description):
    create_cypher = """
    CREATE (:{nodetype} {{Code: "{code}", Description: "{description}" }})
    """
    execute_query(session, create_cypher.format(nodetype=nodetype, code=code,description=description))

def create_fwnode(code, description, contextdesc):
    create_cypher = """
    CREATE (:NISTRequirements {{Code: "{code}", Description: "{description}", Contextual_Description: "{contextdesc}" }})
    """
    execute_query(session, create_cypher.format(code=code,description=description,contextdesc=contextdesc))

#### Loading NIST Framework into Neo4j

In [ ]:
# Creating function nodes and relationship with document
for code, func_desc in zip(df_fw["Function Code"].unique(), df_fw["Function Description"].unique()):
    create_node("Function",code,func_desc)
    create_relationships = """
    MATCH (d:Document), (f:Function)
    WHERE d.Description = "NIST Privacy Framework" AND f.Code = "{code}" 
    CREATE (d)-[:CONTAINS]->(f)
    """
    execute_query(session, create_relationships.format(code=code))

In [ ]:
# Creating category nodes and relationship with function
for func_code in df_fw["Function Code"].unique():
    df_subset = df_fw[df_fw["Function Code"]==func_code]
    for code, cat_desc in zip(df_subset["Category Code"].unique(), df_fw["Category Description"].unique()):
        create_node("Category",code,cat_desc)
        create_relationships = """
        MATCH (f:Function), (c:Category)
        WHERE f.Code = "{func_code}" AND c.Code = "{cat_code}" 
        CREATE (f)-[:DEFINED_BY]->(c)
        """
        execute_query(session, create_relationships.format(func_code=func_code,cat_code=code))

In [ ]:
# Creating FW Requirement nodes and relationship with Category
for cat_code in df_fw["Category Code"].unique():
    #print(cat_code)
    df_subset = df_fw[df_fw["Category Code"]==cat_code]
    for fw_code in df_subset["Framework Requirement Code"].unique():
        #print(fw_code)
        row = df_fw[df_fw["Framework Requirement Code"]==fw_code]
        row.reset_index(inplace=True,drop=True)
        #print(row.shape)
        #print("**")
        create_fwnode(row["Framework Requirement Code"][0],row["Framework Requirement Description"][0],row["Contextualized Framework Requirement Description"][0])
        
        create_relationships = """
        MATCH (c:Category), (fw: NISTRequirements)
        WHERE c.Code = "{cat_code}" AND fw.Code = "{fw_code}" 
        CREATE (c)-[:REQUIRES]->(fw)
        """
        execute_query(session, create_relationships.format(cat_code=cat_code,fw_code=fw_code))

#### Loading DPTM Regulation into Neo4j

In [ ]:
# Creating function nodes and relationship with document
for code, func_desc in zip(df_reg["Function Code"].unique(), df_reg["Function"].unique()):
    create_node("Function",code,func_desc)
    create_relationships = """
    MATCH (d:Document), (f:Function)
    WHERE d.Description = "DPTM Checklist" AND f.Code = "{code}" 
    CREATE (d)-[:CONTAINS]->(f)
    """
    execute_query(session, create_relationships.format(code=code))

In [ ]:
# Creating category nodes and relationship with function
for func_code in df_reg["Function Code"].unique():
    df_subset = df_reg[df_reg["Function Code"]==func_code]
    for code, cat_desc in zip(df_subset["Category Code"].unique(), df_reg["Category"].unique()):
        create_node("Category",code,cat_desc)
        create_relationships = """
        MATCH (f:Function), (c:Category)
        WHERE f.Code = "{func_code}" AND c.Code = "{cat_code}" 
        CREATE (f)-[:DEFINED_BY]->(c)
        """
        execute_query(session, create_relationships.format(func_code=func_code,cat_code=code))

In [ ]:
# Creating Regulation Requirement nodes and relationship with Category
for cat_code in df_reg["Category Code"].unique():
    #print(cat_code)
    df_subset = df_reg[df_reg["Category Code"]==cat_code]
    for reg_code in df_subset["Requirement Code"].unique():
        #print(fw_code)
        row = df_reg[df_reg["Requirement Code"]==reg_code]
        row.reset_index(inplace=True,drop=True)
        #print(row.shape)
        #print("**")
        create_node("DPTMRegRequirement",row["Requirement Code"][0],row["Requirement Description"][0])
        
        create_relationships = """
        MATCH (c:Category), (fw: DPTMRegRequirement)
        WHERE c.Code = "{cat_code}" AND fw.Code = "{reg_code}" 
        CREATE (c)-[:REQUIRES]->(fw)
        """
        execute_query(session, create_relationships.format(cat_code=cat_code,reg_code=reg_code))

#### Creating Mapping connections (Top 5 similarity)

In [ ]:
import ast
dptm_vectors = df_reg["Requirement Description Embedding"].tolist()
nist_vectors = df_fw["Contextualized Framework Requirement Embedding"].tolist()
dptm_vectors = [ast.literal_eval(x) for x in dptm_vectors]
nist_vectors = [ast.literal_eval(x) for x in nist_vectors]
similarity_matrix = cosine_similarity(dptm_vectors,nist_vectors) # We can start looking at similarities from which place one starts

In [ ]:
# Creating a dataframe for which each row is connected to 5 framework requirement
df_new = pd.DataFrame(np.repeat(df_reg.values, 5, axis=0))
df_new.columns = df_reg.columns
df_new.drop(labels=["Policy Documents"],axis=1,inplace=True)
for i in range(len(df_reg)):
#for i in range(2):
    indices_of_interest = np.argsort(similarity_matrix[i,:])[-5:][::-1]
    similarity_score = similarity_matrix[i,indices_of_interest]
    for count,fw_index in enumerate(indices_of_interest):
        matched_framework_reqs = df_fw.iloc[fw_index]
        df_new.loc[i*5+count,'Reg-FW-Similarity-Score'] = similarity_score[count]
        df_new.loc[i*5+count,'Framework Requirement Description'] = matched_framework_reqs['Framework Requirement Description']
        df_new.loc[i*5+count,'Contextualized Framework Requirement Description'] = matched_framework_reqs['Contextualized Framework Requirement Description']
        df_new.loc[i*5+count,'Category Code'] = matched_framework_reqs['Category Code']
        df_new.loc[i*5+count,'Function Code'] = matched_framework_reqs['Function Code']
        df_new.loc[i*5+count,'Framework Requirement Code'] = matched_framework_reqs['Framework Requirement Code']

In [ ]:
for index, row in df_new.iterrows():
    create_relationships = """
    MATCH (fw: NISTRequirements), (reg: DPTMRegRequirement)
    WHERE reg.Code = "{reg_code}" AND fw.Code = "{fw_code}" 
    CREATE (reg)-[m:MAPS_TO]->(fw)
    SET m.SimilarityStrength={simscore} 
    """
    #print(create_relationships.format(reg_code=row["Requirement Code"],fw_code=row["Framework Requirement Code"]))
    execute_query(session, create_relationships.format(reg_code=row["Requirement Code"],fw_code=row["Framework Requirement Code"],simscore=row["Reg-FW-Similarity-Score"]))

#### Creating Mapping connections (Using threshold)

In [ ]:
df_map_strong = df_new[df_new["Reg-FW-Similarity-Score"]>0.55]
df_map_strong.shape

In [ ]:
for index, row in df_map_strong.iterrows():
    create_relationships = """
    MATCH (fw: NISTRequirements), (reg: DPTMRegRequirement)
    WHERE reg.Code = "{reg_code}" AND fw.Code = "{fw_code}" 
    CREATE (reg)-[m:MAPS_TO]->(fw)
    SET m.SimilarityStrength={simscore} 
    """
    #print(create_relationships.format(reg_code=row["Requirement Code"],fw_code=row["Framework Requirement Code"]))
    execute_query(session, create_relationships.format(reg_code=row["Requirement Code"],fw_code=row["Framework Requirement Code"],simscore=row["Reg-FW-Similarity-Score"]))